In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn


import torchvision
import torchvision.transforms as transforms

from resnet_cifar import *
from wide_resnet_cifar import *
from resnext_cifar import *
from densenet_cifar import *
from custom_loader_cifar import *
from orig_cifar import *

import parser

class Parameters:
    def __init__(self, 
                 epoch=200, 
                 batch_size=128, lr=0.1, momentum=0.9, wd=1e-4, print_freq=10, resume="", evaluate=True, ct=10):
        self.epoch = epoch
        self.batch_size = batch_size
        self.lr = lr
        self.momentum = momentum
        self.wd = wd
        self.print_freq = print_freq
        self.resume = resume
        self.evaluate = evaluate
        self.ct = ct
        self.weight_decay = wd
        self.cifar_type = 10

def main1(epoch=200, batch_size=128, lr=0.1, momentum=0.9, wd=1e-4, print_freq=10, resume="", evaluate=False, ct=10):
    
    global args, best_prec
    best_prec = 0
    args = Parameters()
    #args = parser.parse_args()
    args.epoch = epoch
    args.batch_size = batch_size
    args.lr = lr
    args.momentum = momentum
    args.wd = wd
    args.print_freq = print_freq
    args.resume = resume
    args.evaluate = evaluate
    args.ct = ct
    args.start_epoch = 0
    args.epochs = epoch
    
    #print("Hello")
    main2()
    

def main2():
    global args, best_prec
    #print("Hello1")
    '''
    epoch=200, batch_size=128, lr=0.1, momentum=0.9, wd=1e-4, print_freq=10, resume="", evaluate=True, ct=10
    epoch=160, batch_size=128, lr=0.1, momentum=0.9, wd=1e-4, ct=10
    global args, best_prec
    args = parser.parse_args()
    args.epoch = epoch
    args.batch_size = batch_size
    args.lr = lr
    args.momentum = momentum
    args.wd = wd
    args.print_freq = print_freq
    args.resume = resume
    args.evaluate = evaluate
    args.ct = ct
    '''
    
    #global args, best_prec
    #args = parser.parse_args()
    #use_gpu = torch.cuda.is_available()
    use_gpu = False

    # Model building
    print('=> Building model...')
    if use_gpu or True:
        # model can be set to anyone that I have defined in models folder
        # note the model should match to the cifar type !

        model = resnet20_cifar()
        # model = resnet32_cifar()
        # model = resnet44_cifar()
        # model = resnet110_cifar()
        # model = preact_resnet110_cifar()
        # model = resnet164_cifar(num_classes=100)
        # model = resnet1001_cifar(num_classes=100)
        # model = preact_resnet164_cifar(num_classes=100)
        # model = preact_resnet1001_cifar(num_classes=100)

        # model = wide_resnet_cifar(depth=26, width=10, num_classes=100)

        # model = resneXt_cifar(depth=29, cardinality=16, baseWidth=64, num_classes=100)
        
        #model = densenet_BC_cifar(depth=190, k=40, num_classes=100)

        # mkdir a new folder to store the checkpoint and best model
        if not os.path.exists('result'):
            os.makedirs('result')
        fdir = 'result/resnet20_cifar10'
        if not os.path.exists(fdir):
            os.makedirs(fdir)

        # adjust the lr according to the model type
        if isinstance(model, (ResNet_Cifar, PreAct_ResNet_Cifar)):
            model_type = 1
        elif isinstance(model, Wide_ResNet_Cifar):
            model_type = 2
        elif isinstance(model, (ResNeXt_Cifar, DenseNet_Cifar)):
            model_type = 3
        else:
            print('model type unrecognized...')
            return

        #model = nn.DataParallel(model).cuda()
        model = nn.DataParallel(model)
        #criterion = nn.CrossEntropyLoss().cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
        cudnn.benchmark = True
    else:
        print('Cuda is not available!')
        return

    if args.resume:
        if os.path.isfile(args.resume):
            print('=> loading checkpoint "{}"'.format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_prec = checkpoint['best_prec']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    # Data loading and preprocessing
    # CIFAR10
    if args.cifar_type == 10:
        print('=> loading cifar10 data...')
        normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

        
        root_path = "./data_treatment/reverse_2"
        #root_path = "./data"
        #train_dataset = torchvision.datasets.CIFAR10(
        train_dataset = ImagesDataset(
            root=root_path, 
            train=True, 
            download=False,
            transform=transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ]))
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)

        #test_dataset = torchvision.datasets.CIFAR10(
        test_dataset = ImagesDataset(
            root=root_path,
            train=False,
            download=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                normalize,
            ]))
        testloader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
    
    if args.evaluate:
        validate(testloader, model, criterion)
        return

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch, model_type)

        # train for one epoch
        train(trainloader, model, criterion, optimizer, epoch)

        # evaluate on test set
        prec = validate(testloader, model, criterion)

        # remember best precision and save checkpoint
        is_best = prec > best_prec
        best_prec = max(prec,best_prec)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec': best_prec,
            'optimizer': optimizer.state_dict(),
        }, is_best, fdir)


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    
    for i, (input, target) in enumerate(trainloader):
        '''
        print(type(input), input.shape)
        print(type(target), target, len(target), i)
        print(type(input[0][0][0]))
        print(type(target[0]))
        #input = input.double()
        '''
        input = input.double()
        target = target.long()
        
        #print("Here", i)
        # measure data loading time
        data_time.update(time.time() - end)

        #input, target = input.cuda(), target.cuda()
        #input, target = input.cuda(), target.cuda()

        # compute output
        model = model.double()
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))
    


def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            #input, target = input.cuda(), target.cuda()
            input = input.double()
            target = target.long()

            # compute output
            model = model.double()
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))

    return top1.avg


def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch, model_type):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    if model_type == 1:
        if epoch < 80:
            lr = args.lr
        elif epoch < 120:
            lr = args.lr * 0.1
        else:
            lr = args.lr * 0.01
    elif model_type == 2:
        if epoch < 60:
            lr = args.lr
        elif epoch < 120:
            lr = args.lr * 0.2
        elif epoch < 160:
            lr = args.lr * 0.04
        else:
            lr = args.lr * 0.008
    elif model_type == 3:
        if epoch < 150:
            lr = args.lr
        elif epoch < 225:
            lr = args.lr * 0.1
        else:
            lr = args.lr * 0.01
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


In [2]:
#python main.py --epoch 160 --batch-size 128 --lr 0.1 --momentum 0.9 --wd 1e-4 -ct 10
main1(epoch=10, batch_size=128, lr=0.1, momentum=0.9, wd=1e-4, ct=100)
#main1(epoch=2, batch_size=4096, lr=0.1, momentum=0.9, wd=1e-4, ct=10, print_freq=10)
#main()

#, axis = 1

=> Building model...
=> loading cifar10 data...
(3000, 32, 32, 3)
(200, 32, 32, 3)
Epoch: [0][0/24]	Time 6.002 (6.002)	Data 0.111 (0.111)	Loss 2.3627 (2.3627)	Prec 10.156% (10.156%)
Epoch: [0][10/24]	Time 3.295 (3.583)	Data 0.004 (0.014)	Loss 2.3131 (2.3589)	Prec 8.594% (9.446%)
Epoch: [0][20/24]	Time 3.105 (3.405)	Data 0.003 (0.009)	Loss 2.4662 (2.3771)	Prec 8.594% (9.449%)
Test: [0/2]	Time 0.840 (0.840)	Loss 7.6768 (7.6768)	Prec 8.000% (8.000%)
 * Prec 10.500% 
Epoch: [1][0/24]	Time 3.358 (3.358)	Data 0.055 (0.055)	Loss 2.4840 (2.4840)	Prec 10.156% (10.156%)
Epoch: [1][10/24]	Time 3.350 (3.380)	Data 0.004 (0.009)	Loss 2.3178 (2.3840)	Prec 10.938% (11.293%)
Epoch: [1][20/24]	Time 3.079 (3.298)	Data 0.003 (0.006)	Loss 2.3320 (2.3788)	Prec 8.594% (10.417%)
Test: [0/2]	Time 0.881 (0.881)	Loss 2.3834 (2.3834)	Prec 10.000% (10.000%)
 * Prec 9.500% 
Epoch: [2][0/24]	Time 3.999 (3.999)	Data 0.067 (0.067)	Loss 2.3562 (2.3562)	Prec 7.812% (7.812%)
Epoch: [2][10/24]	Time 3.077 (3.416)	Data 0.00

In [ ]:

#New part of the code deals with lens blur (or variable radial blur so as to re-train model)
#N.B. Ealier cell of the notebook does it only 

In [ ]:
#Find depth
import numpy as np
import cv2
from matplotlib import pyplot as plt
#help(cv2.StereoBM_create)

imgL = cv2.imread('stereo/tsukuba_L.png',0)
imgR = cv2.imread('stereo/tsukuba_R.png',0)

#stereo = cv2.createStereoBM(numDisparities=16, blockSize=15)
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()
cv2.imwrite('stereo/tsukuba.png',disparity)


In [ ]:
def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

def load_cfar10_batch_new(cifar10_dataset_folder_path, batch_name):
    with open(cifar10_dataset_folder_path + batch_name, mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

In [ ]:
import pickle, matplotlib.pyplot as plt, os
from PIL import Image

#a, b = load_cfar10_batch("./data/", 1)
#print(len(b))
#plt.imshow(a[1])
#plt.save(a[1], '1.jpg')

for batch in [1, 2, 3, 4, 5]:
    a, b = load_cfar10_batch("./data/", 1)
    directory = "CIFAR_extracted_data_{}".format(batch)
    if not os.path.exists(directory):
        os.makedirs(directory)

    for idx in range(1, 10001):
        img = Image.fromarray(a[1])
        img.save('./CIFAR_extracted_data_{}/{}.jpg'.format(batch, idx))


In [ ]:
from scipy import misc
misc.imsave('my.jpg', a[1])

In [ ]:
for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
    a, b = load_cfar10_batch_new("./data/", batch)
    directory = "./images/orig/CIFAR_{}".format(batch)
    if not os.path.exists(directory):
        os.makedirs(directory)

    for idx in range(1, 10001):
        img = Image.fromarray(a[1])
        img.save('./images/orig/CIFAR_{}/{}.jpg'.format(batch, idx))

In [ ]:
def load_cfar10_batch_new(cifar10_dataset_folder_path, batch_name):
    with open(cifar10_dataset_folder_path + batch_name, mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data']
    #print(features.shape, type(features))
    features = features.reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    print(features.shape, type(features))
    labels = batch['labels']
        
    return features, labels

for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
    arr, labels = load_cfar10_batch_new("./data/", batch)
    directory = "./images/orig/CIFAR_{}".format(batch)
    
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)
    '''

    for idx in range(1, 10001):
        continue
        #img = Image.fromarray(a[1])
        #img.save('./images/orig/CIFAR_{}/{}.jpg'.format(batch, idx))
        

In [ ]:
import numpy as np
im = np.array(Image.open('1.jpg'))
print(im.shape, im.dtype)
im1 = im

In [ ]:
for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
    print(batch)
    arr, labels = load_cfar10_batch_new("./data/", batch)
    directory = "./images/orig/CIFAR_{}".format(batch)
    
    data_new = {}
    data_new['data'] = np.empty([10000, 3072])
    data_new['labels'] = labels
    
    for idx in range(1, 10001):
        im = Image.open('./images/orig/CIFAR_{}/{}.jpg'.format(batch, idx))
        im = (np.array(im))
        r = im[:,:,0].flatten()
        g = im[:,:,1].flatten()
        b = im[:,:,2].flatten()
        
        data_new['data'][idx - 1] = np.array(list(r) + list(g) + list(b), dtype=np.uint8)
    
    print(data_new['data'].shape, type(data_new['data']))
    
    directory_new = "./data_new/cifar-10-batches-py/"
    if not os.path.exists(directory_new):
        os.makedirs(directory_new)
        
    with open(directory_new + batch, 'wb') as file:
        pickle.dump(data_new, file)

    with open(directory_new + batch, 'rb') as file:
        data_new_rd = pickle.load(file, encoding='latin1')

    #print(data_new['data'] == data_new_rd['data'], data_new['labels'] == data_new_rd['labels'])


In [ ]:
for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
    print(batch)
    arr, labels = load_cfar10_batch_new("./data/", batch)
    arr, labels = load_cfar10_batch_new("./data_new/", batch)


In [ ]:
for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
    arr, labels = load_cfar10_batch_new("./data/", batch)
    directory = "./images/orig/CIFAR_{}".format(batch)
    
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)
    '''

    for idx in range(1, 10001):
        continue
        im = Image.open('./images/orig/CIFAR_{}/{}.jpg'.format(batch, idx))
        im = (np.array(im))
        
        
im = Image.open('1.jpg')
im = (np.array(im))

r = im[:,:,0].flatten()
g = im[:,:,1].flatten()
b = im[:,:,2].flatten()
#label = [1]
data_new = {}
data_new['data'] = np.empty([10000, 3072])
print(data_new['data'].shape, type(data_new['data']))

#out = np.array(list(label) + list(r) + list(g) + list(b),np.uint8)
data_new['data'][0] = np.array(list(r) + list(g) + list(b), dtype=np.uint8)
print(data_new['data'].shape, type(data_new['data']))
print(data_new['data'][0].shape)
#out.tofile("out.bin")


In [ ]:
import numpy as np, pickle, os
from PIL import Image

def load_cfar10_batch_new(cifar10_dataset_folder_path, batch_name):
    with open(cifar10_dataset_folder_path + batch_name, mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data']
    #print(features.shape, type(features))
    features = features.reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    print(features.shape, type(features))
    labels = batch['labels']
        
    return features, labels

for method in ['reverse']:
    for route in range(2, 3):
        
        for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
            arr, labels = load_cfar10_batch_new("./data/", batch)
            directory = "./images/new/CIFAR_{}_{}_{}".format(batch, method, route)
            print(directory)

            #'''
            data_new = {}
            data_new['data'] = np.empty([10000, 3072])
            data_new['labels'] = labels
            print(len(labels))

            for idx in range(1, 10001):
                im = Image.open("{}/{}.jpg".format(directory, idx))
                #./images/orig/CIFAR_{}/{}.jpg'.format(batch, idx))
                im = (np.array(im))
                r = im[:,:,0].flatten()
                g = im[:,:,1].flatten()
                b = im[:,:,2].flatten()

                data_new['data'][idx - 1] = np.array(list(r) + list(g) + list(b), dtype=np.uint8)
                #print(data_new['data'][idx - 1].shape)
                #data_new['data'][idx - 1] = np.array(list(r) + list(g) + list(b), axis=1)
                #print(np.concatenate((list(r), list(g), list(b)), axis = 0).shape)
                data_new['data'][idx - 1] = np.concatenate((list(r), list(g), list(b)), axis = 0)
                #print(data_new['data'][idx - 1].shape)

            print(data_new['data'].shape, type(data_new['data']))

            #'''
            directory_new = "./data_treatment/{}_{}/cifar-10-batches-py/".format(method, route)
            if not os.path.exists(directory_new):
                os.makedirs(directory_new)

            #'''
            with open(directory_new + batch, 'wb') as file:
                pickle.dump(data_new, file)
            #'''
                
            #'''
            
            #'''
            with open(directory_new + batch, 'rb') as file:
                data_new_rd = pickle.load(file, encoding='latin1')
            #'''

            #print(data_new['data'] == data_new_rd['data'], data_new['labels'] == data_new_rd['labels'])
            #print(data_new['data'] == data_new_rd['data'])
            #print(data_new['labels'] == data_new_rd['labels'])
            #break
        #break
    #break


In [ ]:
for method in ['reverse']:
    data_all = np.array([], dtype=np.int64).reshape(0, 32, 32, 3)
    labels_all = []

    for route in range(2, 3):
        for batch in ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']:
            arr, labels = load_cfar10_batch_new("./data/", batch)
            labels_all = labels_all + labels
            print(arr.shape)
            data_all = np.vstack((data_all, arr))
            
            print(len(labels_all), data_all.shape)